# Eliminar encabezado y sumarios

Elimino estos datos del campo boletin

In [1]:
from pymongo import MongoClient, UpdateOne
import re
from pprint import pprint

In [2]:
uri = open('mongo_uri.txt', 'r', encoding='utf-8')
client = MongoClient(uri)
pprint(client)
db = client.boletines_db
pprint(db.boletines.find_one())

MongoClient(host=['clustershido-shard-00-01-uxkjr.gcp.mongodb.net:27017', 'clustershido-shard-00-02-uxkjr.gcp.mongodb.net:27017', 'clustershido-shard-00-00-uxkjr.gcp.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='ClusterShido-shard-0', ssl=True)
{'_id': ObjectId('5ec3089dd380e649eb619ce9'),
 'boletin': 'Juntos haremos historia\n'
            'BOLETÍN CAMPAÑA-009\n'
            'PIDE LÓPEZ OBRADOR A TRUMP QUE NO MALTRATE A LOS MEXICANOS, '
            '“QUEREMOS RESPETO MUTUO”\n'
            '- NI LAS AMENAZAS, NI LOS MILITARES A LA FRONTERA EVITARAN QUE '
            'MÉXICO QUIERA TENER UNA RELACIÓN DE AMISTAD CON EU: AMLO\n'
            'MATAMOROS, TAMAULIPAS, 06 DE ABRIL DE 2018.- Andrés Manuel López '
            'Obrador manifestó que México y Estados Unidos “somos pueblos que '
            'estamos hermanados, los gobiernos no tienen por qué separar a los '
            'pueblos, nosotros le p

## AMLO

El contenido principal de los boletines de AMLO inicia con una cadena con el siguiente formato:

```
<localidad>, <estado>, <fecha> .- <inicia_boletin>
```

Así que, sólo debemos tomar lo que queda después de los caracteres .-

In [7]:
cursor = db.boletines.find(
    {'candidato': 'AMLO'}, # Filtro
    {'boletin':1} # Proyección
).sort('fecha')

In [11]:
patron = re.compile(r'(.+)\.- ')
n_matched = 0
updates = []
for boletin in cursor:
    contenido = boletin['boletin']
    match = patron.search(contenido)
    print(boletin['_id'])
    if match:
        n_matched += 1
        print(match[0])
#         print(match.span(0))
        print(f'El patrón termina en pos {match.span(0)[1]}')
#         print(f'{contenido[match.span(0)[1]:]}\n')
        updates.append(UpdateOne(
            {'_id': boletin['_id']},
            {'$set': {
                'boletin':contenido[match.span(0)[1]:]
            }}
        ))
        print(contenido[match.span(0)[1]:match.span(0)[1]+15])
    else:
        print('No hay coincidencia')
    print()
cursor.rewind()
n_matched

5ec3089dd380e649eb619ce3
SANTA CATARINA, NUEVO LEÓN, 02 DE ABRIL DE 2018.- 
El patrón termina en pos 549
Andrés Manuel L

5ec3089dd380e649eb619ce4
SAN PEDRO, COAHUILA, 04 DE ABRIL DE 2018.- 
El patrón termina en pos 586
El candidato de

5ec3089dd380e649eb619ce5
MONCLOVA, COAHUILA, 04 DE ABRIL DE 2018.- 
El patrón termina en pos 421
Andrés Manuel L

5ec3089dd380e649eb619ce6
EN PIEDRAS NEGRAS, COAHUILA, 04 DE ABRIL DE 2018.- 
El patrón termina en pos 304
Andrés Manuel L

5ec3089dd380e649eb619ce7
NUEVO LAREDO, TAMAULIPAS, 05 DE ABRIL DE 2018.- 
El patrón termina en pos 626
Andrés Manuel L

5ec3089dd380e649eb619ce8
REYNOSA, TAMAULIPAS, 05 DE ABRIL DE 2018.- 
El patrón termina en pos 426
Andrés Manuel L

5ec3089dd380e649eb619ce9
MATAMOROS, TAMAULIPAS, 06 DE ABRIL DE 2018.- 
El patrón termina en pos 294
Andrés Manuel L

5ec3089dd380e649eb619cea
IRAPUATO, GUANAJUATO 07 DE ABRIL DE 2018.- 
El patrón termina en pos 430
Andrés Manuel L

5ec3089dd380e649eb619ceb
LAGOS DE MORENO, JALISCO 08 DE ABR

84

In [14]:
res = db.boletines.bulk_write(updates)
pprint(res.bulk_api_result)

{'nInserted': 0,
 'nMatched': 84,
 'nModified': 84,
 'nRemoved': 0,
 'nUpserted': 0,
 'upserted': [],
 'writeConcernErrors': [],
 'writeErrors': []}


## JAMK

Los encabezados, sumarios, fechas e identificadores de boletin de JAMK cumplen con la siguiente regex: `[^\.\?:!]$`.

In [21]:
cursor = db.boletines.find(
    {'candidato': 'JAMK'}, # Filtro
    {'boletin':1} # Proyección
).sort('fecha')

patron = re.compile(r'[^\.\?:!]$')

In [24]:
updates = []
for boletin in cursor:
    contenido = boletin['boletin'].split('\n')
    n_matched = 0
    for i, sent in enumerate(contenido):
        if i>=8:
            break
        match = patron.search(sent.strip())
        if match:
            n_matched += 1
#             print(sent.strip())
    #         print(match.span(0))
#             print(f'El patrón termina en pos {match.span(0)[1]}')
    #         print(f'{contenido[match.span(0)[1]:]}\n')
        else:
            break
#             print('No hay patrón en', fname)
    if n_matched >0:
        updates.append(UpdateOne(
            {'_id': boletin['_id']},
            {'$set': {
                'boletin':'\n'.join(contenido[n_matched:])
            }}
        ))
    print(f'Líneas eliminadas: {n_matched}')
cursor.rewind()

Líneas eliminadas: 4
Líneas eliminadas: 3
Líneas eliminadas: 6
Líneas eliminadas: 3
Líneas eliminadas: 3
Líneas eliminadas: 4
Líneas eliminadas: 3
Líneas eliminadas: 3
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 6
Líneas eliminadas: 4
Líneas eliminadas: 3
Líneas eliminadas: 5
Líneas eliminadas: 4
Líneas eliminadas: 5
Líneas eliminadas: 3
Líneas eliminadas: 6
Líneas eliminadas: 6
Líneas eliminadas: 5
Líneas eliminadas: 4
Líneas eliminadas: 5
Líneas eliminadas: 4
Líneas eliminadas: 6
Líneas eliminadas: 6
Líneas eliminadas: 3
Líneas eliminadas: 6
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 6
Líneas eliminadas: 3
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 3
Líneas eliminadas: 3
Líneas eliminadas: 3
Líneas eliminadas: 6
Líneas eliminadas: 5
Líneas eliminadas: 3
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 3
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas elimin

In [25]:
res = db.boletines.bulk_write(updates)
pprint(res.bulk_api_result)

{'nInserted': 0,
 'nMatched': 147,
 'nModified': 147,
 'nRemoved': 0,
 'nUpserted': 0,
 'upserted': [],
 'writeConcernErrors': [],
 'writeErrors': []}


## RAC

Los encabezados, fechas e identificadores de boletin de RAC cumplen con la siguiente regex: `[^\.\?:!]$`.

Los sumarios con la regex: `^[(* )(\u2022 )]`, donde \u2022 es el caracter •

In [35]:
cursor = db.boletines.find(
    {'candidato': 'RAC'}, # Filtro
    {'boletin':1} # Proyección
).sort('fecha')

patron = re.compile(r'([^\.\?:!]$)|(^[(* )(\u2022 )])')

In [37]:
updates = []
for boletin in cursor:
    contenido = boletin['boletin'].split('\n')
    n_matched = 0
    for i, sent in enumerate(contenido):
        if i>=8:
            break
        match = patron.search(sent.strip())
        if match:
            n_matched += 1
        else:
            break # En cuanto no encuentre coincidencia, sale del bucle
    if n_matched >0:
#         print('\n'.join(contenido[n_matched:n_matched+2]))
#         print('\n'.join(contenido[:n_matched]))
        updates.append(UpdateOne(
            {'_id': boletin['_id']},
            {'$set': {
                'boletin':'\n'.join(contenido[n_matched:])
            }}
        ))
    print(f'Líneas eliminadas: {n_matched}')
cursor.rewind()

Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 4
Líneas eliminadas: 4
Líneas eliminadas: 4
Líneas eliminadas: 4
Líneas eliminadas: 6
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 4
Líneas eliminadas: 5
Líneas eliminadas: 3
Líneas eliminadas: 5
Líneas eliminadas: 3
Líneas eliminadas: 5
Líneas eliminadas: 4
Líneas eliminadas: 4
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 4
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 4
Líneas eliminadas: 4
Líneas eliminadas: 3
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 4
Líneas eliminadas: 5
Líneas eliminadas: 4
Líneas eliminadas: 4
Líneas eliminadas: 4
Líneas eliminadas: 3
Líneas eliminadas: 4
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas eliminadas: 4
Líneas eliminadas: 5
Líneas eliminadas: 5
Líneas elimin

In [38]:
res = db.boletines.bulk_write(updates)
pprint(res.bulk_api_result)

{'nInserted': 0,
 'nMatched': 116,
 'nModified': 116,
 'nRemoved': 0,
 'nUpserted': 0,
 'upserted': [],
 'writeConcernErrors': [],
 'writeErrors': []}


In [39]:
client.close()